# Collect resources

## From GitHub

In [3]:
!git clone https://github.com/mrok273/Qiita ../data/mrok273/Qiita || git -C ../data/mrok273/Qiita pull


Cloning into '../data/mrok273/Qiita'...
Updating files:  69% (1780/2557)
Updating files:  70% (1790/2557)
Updating files:  71% (1816/2557)
Updating files:  72% (1842/2557)
Updating files:  73% (1867/2557)
Updating files:  74% (1893/2557)
Updating files:  75% (1918/2557)
Updating files:  76% (1944/2557)
Updating files:  77% (1969/2557)
Updating files:  78% (1995/2557)
Updating files:  79% (2021/2557)
Updating files:  80% (2046/2557)
Updating files:  81% (2072/2557)
Updating files:  82% (2097/2557)
Updating files:  83% (2123/2557)
Updating files:  84% (2148/2557)
Updating files:  85% (2174/2557)
Updating files:  86% (2200/2557)
Updating files:  87% (2225/2557)
Updating files:  88% (2251/2557)
Updating files:  89% (2276/2557)
Updating files:  90% (2302/2557)
Updating files:  91% (2327/2557)
Updating files:  92% (2353/2557)
Updating files:  93% (2379/2557)
Updating files:  94% (2404/2557)
Updating files:  94% (2406/2557)
Updating files:  95% (2430/2557)
Updating files:  96% (2455/2557)
Upd

## From web

- Install firefox
- Open web page
- Bulk save image (See [How to Save All the Images on a Web Page in Firefox Browser](https://www.journeybytes.com/bulk-save-images-using-firefox/))

## From YouTube

In [1]:
# Official paldeck
!yt-dlp --postprocessor-args "-ss 00:00:00 -t 00:00:05" -o "../data/video/pocketpair/%(title)s-%(id)s-5s.%(ext)s" https://www.youtube.com/playlist?list=PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s
[youtube:tab] PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s: Downloading webpage
[youtube:tab] PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Paldeck
[youtube:tab] PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s page 1: Downloading API JSON
[youtube:tab] PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s page 1: Downloading API JSON
[youtube:tab] PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s page 1: Downloading API JSON
[youtube:tab] PLptNv_Fxn9idzsTRulWNmLYKWgKhqKI5s page 1: Downloading API JSON
[youtube:tab] Playlist Paldeck: Downloading 57 items of 57
[download] Downloading item 1 of 57
[youtube] Extracting URL: https://www.youtube.com/watch?v=D0Ds7F9If6o
[youtube] D0Ds7F9If6o: Downloading webpage
[youtube] D0Ds7F9If6o: Downloading ios player API JSON
[youtube] D0Ds7F9If6o: Downloading android player API JSON
[youtube] D0Ds7F9If6o: Downloading m3u8 inform

In [10]:
import os
import re

directory = "../data/video/pocketpair"
for filename in os.listdir(directory):
    match = re.search(r'[Pp]aldeck.*[Nn]o.(\d+).*.webm', filename)
    paldeck_no, = match.groups() if match else [None]
    if paldeck_no is None:
        continue
    new_filename = f"paldeck_no{paldeck_no.zfill(3)}.webm"
    os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))

In [12]:
!ffprobe -v error -select_streams v:0 -show_entries stream=width,height,r_frame_rate -of csv=s=x:p=0 "../data/video/pocketpair/paldeck_no001.webm"

1920x1080x60/1


In [21]:
import sys
sys.path.append('..')

import os
import subprocess

from datetime import datetime
from src.pipeline import *

input_dir = "../data/video/pocketpair"
output_dir = "../data/raw/pocketpair"

for root, dirs, files in os.walk(input_dir):
    for filename in files:
        full_input_path = os.path.join(root, filename)
        filename_without_ext, _ext = os.path.splitext(filename)
        output_subdir = os.path.join(output_dir, filename_without_ext)
        os.makedirs(output_subdir, exist_ok=True)

        output_pattern = os.path.join(output_subdir, "frame_%05d.png")
        command = ['ffmpeg', '-hwaccel', 'cuda', '-i', full_input_path, '-vf', 'fps=12', output_pattern]
        subprocess.run(command, check=True)
        
        for root_out, _, files_out in os.walk(output_subdir):
            for filename_out in files_out:
                full_output_path = os.path.join(root_out, filename_out)
                raw_dir = data_dir(Step.raw.value)
                metadata = Metadata(
                    bucket=raw_dir,
                    path=os.path.relpath(full_output_path, raw_dir),
                    step=Step.raw,
                    label=Label.pal,
                    created_at=datetime.utcnow()
                )
                create_metadata(metadata)
        